Ноутбук для генерации текстов разными способами, используя GPT2 модель. 

Разные методы для сэмплирования:


*   Argmax
*   Temperature
*   Nucleus sampling
*   Top K
*   Beam Search



In [0]:
!pip install transformers

In [0]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from tqdm import tqdm
import re
import torch
import numpy as np

Сюда подается текстовый файл с предложениями для генерации. Каждое предложение с новой строки. 

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

idd = '1Oou1SfdpxBIle3F3iGTaVU4aU2mkRMXA' #Sentences for gpt2 generation
downloaded_ = drive.CreateFile({'id':idd}) 
downloaded_.GetContentFile('texts_to_gpt2')

In [0]:
def open_file(file):
    with open(file, 'r', encoding='utf-8') as f:
        texts = [line for line in f.readlines()]
    return texts

def save_file(texts, filename):
    with open(filename, 'w', encoding='utf-8') as f:
            for item in texts:
                f.write('{}\n'.format(item))

In [0]:
texts = open_file('texts_to_gpt2')

In [0]:
MAX_LENGTH = 30
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Generation stuff

In [0]:
def generate_fake(texts, sampling_type, fake=[], tokenizer=tokenizer, model=model):
    """
    Функция для генерации гекстов в зависимости от типа сэмплирования. 
    sampling type: tuple (sampling_type : value)
    Записывает фейковые тексты в отдельный файл. (В DataGen_Class есть вариант еще и с обработкой real)
    Возвращает список фейковых текстов.
    """

    i = 0
    for ind, el in tqdm(enumerate(texts)):
        sent = generate_sentence(el, model, tokenizer, sampling_type)            
        sent = re.sub(r'\n', '', sent)
        fake.append(sent)

        if (i % 1000 == 0) and (i > 0):
            print('{} out of {} are not found in emb dict'.format(i, ind))

    with open('fake_{}.txt'.format(sampling_type[0]), 'w', encoding='utf-8') as f:    
        for item in fake:
            f.write('{}\n'.format(item))
    
    return fake

In [0]:
def generate_sentence(sentence, model, tokenizer, sampling_type, max_length=MAX_LENGTH):    
    """
    Функция генерирует предложение в зависимости от типа сэмплирования
    """
    context = torch.tensor([tokenizer.encode(sentence)])
    max_length += context.size()[-1]
    
    if sampling_type[0] == 'beam_search':
        outputs = model.generate(input_ids=context, max_length=max_length,
                                 do_sample=True, num_beams=sampling_type[1])
    elif sampling_type[0] == 'temperature':
        outputs = model.generate(input_ids=context, max_length=max_length,
                                 do_sample=True, temperature=sampling_type[1])
    elif sampling_type[0] == 'top_k':
        outputs = model.generate(input_ids=context, max_length=max_length,
                                 do_sample=True, temperature=sampling_type[1])
    elif sampling_type[0] == 'nucleus':
        outputs = model.generate(input_ids=context, max_length=max_length,
                                 do_sample=True, top_p=sampling_type[1])
    else: # Иначе аргмакс
        outputs = model.generate(input_ids=context, max_length=max_length, do_sample=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Проблемка, пробельчик не добавляется перед сгенерированным кусочком.

In [0]:
argmax_fake = generate_fake(texts[:10], ('argmax', 23), fake=[], tokenizer=tokenizer, model=model)

10it [00:20,  2.06s/it]


In [0]:
beam_search_fake = generate_fake(texts[:10], ('beam_search', 3), fake=[], tokenizer=tokenizer, model=model)

10it [00:41,  4.28s/it]


In [0]:
temperature_fake = generate_fake(texts[:10], ('temperature', 0.8), fake=[], tokenizer=tokenizer, model=model)

10it [00:20,  2.07s/it]


In [0]:
top_k_fake = generate_fake(texts[:10], ('top_k', 30), fake=[], tokenizer=tokenizer, model=model)

10it [00:19,  1.78s/it]


In [0]:
nucleus_fake = generate_fake(texts[:10], ('nucleus', 0.9), fake=[], tokenizer=tokenizer, model=model)

10it [00:21,  2.18s/it]


In [0]:
len(argmax_fake), len(beam_search_fake), len(temperature_fake), len(top_k_fake), len(nucleus_fake)

(10, 10, 10, 10, 10)

In [0]:
argmax_tuple =  ('Argmax', argmax_fake)
beam_search_tuple = ('Beam Search', beam_search_fake)
temperature_tuple = ('Temperature', temperature_fake)
top_k_tuple = ('Top K', top_k_fake)
nucleus_tuple = ('Nucleus', nucleus_fake)

# Comparison

In [0]:
def compare(real, *args):
    for ind, el in enumerate(real):
        print('REAL TEXT: {}'.format(el))
        
        for type_fake, fake_list in args:
            for fake_text in fake_list:
                if fake_text.startswith(el[:20]):
                    print('GENERATED TEXT ({}): {}'.format(type_fake, fake_text))
        print('=======================================')

In [0]:
compare(texts[:10], argmax_tuple, temperature_tuple, top_k_tuple, beam_search_tuple, nucleus_tuple)

REAL TEXT: HAMBURG, Germany, June 3  As he left the soccer field after a club match in the eastern German city of Halle on March 25, the Nigerian forward Adebowale Ogungbure was spit upon, jeered with racial remarks and mocked with monkey noises.

GENERATED TEXT (Argmax): HAMBURG, Germany, June 3  As he left the soccer field after a club match in the eastern German city of Halle on March 25, the Nigerian forward Adebowale Ogungbure was spit upon, jeered with racial remarks and mocked with monkey noises.Ogungbure, who was born in Nigeria, was arrested in Germany on March 25 and charged with "racial discrimination" after!
GENERATED TEXT (Temperature): HAMBURG, Germany, June 3  As he left the soccer field after a club match in the eastern German city of Halle on March 25, the Nigerian forward Adebowale Ogungbure was spit upon, jeered with racial remarks and mocked with monkey noises.The incident sparked a national debate, with many Nigerian supporters condemning the act and others call

# Moie

In [0]:
to_generate = ['And if you go I wanna go with you and if you die I wanna die with you take your hand and go away',
               "I'm going back to 505 ",
               "Coronavirus is going to kill us all?",
               "Yesterday one girl got her diploma and got so drunk",
               "Real Madrid is"]

In [0]:
my_argmax_fake = generate_fake(to_generate, ('argmax', 23), fake=[], tokenizer=tokenizer, model=model)
my_beam_search_fake = generate_fake(to_generate, ('beam_search', 3), fake=[], tokenizer=tokenizer, model=model)
my_temperature_fake = generate_fake(to_generate, ('temperature', 0.8), fake=[], tokenizer=tokenizer, model=model)
my_top_k_fake = generate_fake(to_generate, ('top_k', 30), fake=[], tokenizer=tokenizer, model=model)
my_nucleus_fake = generate_fake(to_generate, ('nucleus', 0.9), fake=[], tokenizer=tokenizer, model=model)

my_argmax_tuple =  ('Argmax', my_argmax_fake)
my_beam_search_tuple = ('Beam Search', my_beam_search_fake)
my_temperature_tuple = ('Temperature', my_temperature_fake)
my_top_k_tuple = ('Top K', my_top_k_fake)
my_nucleus_tuple = ('Nucleus', my_nucleus_fake)


compare(to_generate, my_argmax_tuple, my_beam_search_tuple, my_temperature_tuple, my_top_k_tuple, my_nucleus_tuple)

5it [00:09,  1.93s/it]
5it [00:18,  3.79s/it]
5it [00:09,  1.98s/it]
5it [00:09,  1.96s/it]
5it [00:10,  2.06s/it]

REAL TEXT: And if you go I wanna go with you and if you die I wanna die with you take your hand and go away
GENERATED TEXT (Argmax): And if you go I wanna go with you and if you die I wanna die with you take your hand and go away.I'm not going to let you go.I'm not going to let you go.I'm not going!
GENERATED TEXT (Beam Search): And if you go I wanna go with you and if you die I wanna die with you take your hand and go away as fast as I can" (Nathaniel 4:12). This quote could very well be a reference to what Nietzsche is saying in the!
GENERATED TEXT (Temperature): And if you go I wanna go with you and if you die I wanna die with you take your hand and go away""I'm going with you for the sake of those who love me""I'm going to keep you in my!
GENERATED TEXT (Top K): And if you go I wanna go with you and if you die I wanna die with you take your hand and go away, man - then leave now..." "...We do so willingly?" "- you, me I take. He said...so you leave it up at!
GENERATED TEXT (Nucleus